In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier


## Next Step

Target Encoding 

Optuna

Ensumble 

## Learning trend for the model

Benchmark: 
    
    310.88949952769605 (val) 360.22544 (test)

Add Word count for description and Loan use:
    
    302.439362173603 (val) 351.02701 (test)
    
Add tags count and tags null checker

    287.8709482227993 (val) 339.53605 (test)
 
Add BERT for description translated and loan use

    287.8878845648283 (val) 333.04836 (test)
    
Add 5 fold cross validarion 

    280.73464 (val) 327.92202 (test)

In [41]:
train = pd.read_csv('after_processing.csv')
y = pd.read_csv('y.csv')
test = pd.read_csv('test.csv')

In [42]:
train_columns = ["ORIGINAL_LANGUAGE", "ACTIVITY_NAME", "SECTOR_NAME", "COUNTRY_CODE","CURRENCY_POLICY",
       "CURRENCY","REPAYMENT_INTERVAL", "DISTRIBUTION_MODEL"]


In [43]:
test_X = test[train_columns]

for column in test_X.columns:
    le = LabelEncoder()
    target_column = test_X[column]
    le.fit(target_column)
    label_encoded_column = le.transform(target_column)
    test_X[column] = pd.Series(label_encoded_column).astype("int")



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
# Count the number of the translated description
test['count_description_translated'] = test['DESCRIPTION_TRANSLATED'].apply(lambda x: x.split()) 
test['count_description_translated'] = test['count_description_translated'].apply(lambda x: len(x)) 
test['count_LOAN_USE'] = test['LOAN_USE'].apply(lambda x: x.split()) 
test['count_LOAN_USE'] = test['count_LOAN_USE'].apply(lambda x: len(x)) 
test['TAGS_NULL'] = test['TAGS'].isnull() * 1

def change_tags(x): 
    x = x.split(',') 
    return x 
test['TAGS'] = test['TAGS'].fillna('NAN')
test['change_TAGS'] = test['TAGS'].apply(change_tags)
test['count_tags'] = test['change_TAGS'].apply(lambda x: len(x))

test.head()

,LOAN_ID,ORIGINAL_LANGUAGE,DESCRIPTION,DESCRIPTION_TRANSLATED,IMAGE_ID,ACTIVITY_NAME,SECTOR_NAME,LOAN_USE,COUNTRY_CODE,COUNTRY_NAME,...,CURRENCY_EXCHANGE_COVERAGE_RATE,CURRENCY,TAGS,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL,count_description_translated,count_LOAN_USE,TAGS_NULL,change_TAGS,count_tags
0,2041445,English,Marcela is 69 years old and married with ten c...,Marcela is 69 years old and married with ten c...,4051101,General Store,Retail,to buy items to sell like canned goods and per...,PH,Philippines,...,NaN,PHP,NAN,monthly,field_partner,77,12,1,[NAN],1
1,1944435,English,Roselia is 48 years old and has five children....,Roselia is 48 years old and has five children....,3410523,Pigs,Agriculture,to buy feeds and other supplies to raise her pig,PH,Philippines,...,NaN,PHP,"#Animals, #Repeat Borrower, #Schooling, #Woman...",monthly,field_partner,92,10,0,"[#Animals, #Repeat Borrower, #Schooling, #W...",5
2,2083354,English,"Ma. Marebil is a single woman, 40 years old wi...","Ma. Marebil is a single woman, 40 years old wi...",4146690,Clothing Sales,Clothing,to buy additional stock of clothes and dresses...,PH,Philippines,...,NaN,PHP,"#Parent, #Single Parent, #Woman-Owned Business",monthly,field_partner,64,10,0,"[#Parent, #Single Parent, #Woman-Owned Busin...",3
3,1993565,English,"Good day, lenders! Meet one of KBMI’s clients,...","Good day, lenders! Meet one of KBMI’s clients,...",3945982,Food,Food,to buy more foods to grow her business.,ID,Indonesia,...,0.1,IDR,"#Woman-Owned Business, #Schooling, #Elderly, #...",monthly,field_partner,153,8,0,"[#Woman-Owned Business, #Schooling, #Elderly...",4
4,2064272,English,Rosemarie is a married woman with two children...,Rosemarie is a married woman with two children...,4114040,Food,Food,to buy ingredients for her food production bus...,PH,Philippines,...,NaN,PHP,NAN,monthly,field_partner,80,8,1,[NAN],1


In [45]:
numerical = test[['count_description_translated', 'count_LOAN_USE','TAGS_NULL','count_tags']]
test = pd.concat([test_X, numerical], axis=1)

## Add BERT for description and loan use

In [46]:
translated_test = pd.read_csv('translated_test.csv')
test = pd.concat([test,translated_test],axis=1)
loan_use_test = pd.read_csv('loan_use_test.csv')
test = pd.concat([test,loan_use_test],axis=1)

In [47]:
test.head()

,ORIGINAL_LANGUAGE,ACTIVITY_NAME,SECTOR_NAME,COUNTRY_CODE,CURRENCY_POLICY,CURRENCY,REPAYMENT_INTERVAL,DISTRIBUTION_MODEL,count_description_translated,count_LOAN_USE,...,bert_pca_vecs=010,bert_pca_vecs=011,bert_pca_vecs=012,bert_pca_vecs=013,bert_pca_vecs=014,bert_pca_vecs=015,bert_pca_vecs=016,bert_pca_vecs=017,bert_pca_vecs=018,bert_pca_vecs=019
0,0,70,11,39,1,34,2,1,77,12,...,-1.180859,1.098655,0.464951,0.393744,-0.645396,-1.077443,-0.238121,0.224336,0.411929,-0.211481
1,0,118,0,39,1,34,2,1,92,10,...,-0.291154,-0.337270,-1.208851,0.337044,0.688946,0.001357,0.353752,0.187292,-0.446153,0.721414
2,0,34,2,39,1,34,2,1,64,10,...,0.065822,-0.312215,0.608632,1.144690,0.846098,-0.369058,-0.394486,-0.237144,-0.536992,0.080827
3,0,62,6,18,0,15,2,1,153,8,...,1.214276,-0.215234,0.167419,-0.617937,-0.284667,0.183852,0.263447,0.144234,-0.365589,-0.824010
4,0,62,6,39,1,34,2,1,80,8,...,0.824629,1.127113,0.161120,-0.739084,0.251277,0.184213,-0.228298,0.349612,0.097553,0.453247


In [48]:

model_params = {
    'n_estimators': 1000,
    'objective': 'mae',
    'learning_rate': 0.1,
    'random_state': 42,
    'n_jobs': -1,
}

fit_params = {
    'early_stopping_rounds': 100,
    'verbose': False
}

In [49]:
def make_kf(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    return list(kf.split(X))

In [50]:
def train_cv(X, y, model, model_params, fit_params, cv, folds):
    oof = []; va_idxes = []; scores = []; models = {}

    train_x, train_y = X.values, y.values
    fold_idx = cv(train_x, train_y, n_splits=folds)

    for fold, (tr_idx, va_idx) in enumerate(fold_idx):
        tr_x, va_x = train_x[tr_idx], train_x[va_idx]
        tr_y, va_y = train_y[tr_idx], train_y[va_idx]
        va_idxes.append(va_idx)

        est = model(**model_params)

        est.fit(tr_x, np.log1p(tr_y),
               eval_set=[[va_x, np.log1p(va_y)]],
               **fit_params)
        
        model_name = f'LGBM_FOLD{fold}'
        models[model_name] = est

        preds = est.predict(va_x)
        preds = np.expm1(preds)
        oof.append(preds)

        score = mean_absolute_error(va_y, preds)
        scores.append(score)
        print(f'FOLD: {fold+1}, SCORE: {score:.5f}')

    va_idxes = np.concatenate(va_idxes)
    oof = np.concatenate(oof)
    order = np.argsort(va_idxes)
    oof = oof[order]

    oof_score = mean_absolute_error(train_y, oof)
    print(f'oof score: {oof_score:.5f}\n')
    return oof, models

In [51]:
%%time
oof, models = train_cv(train, y['LOAN_AMOUNT'],
                        lgb.LGBMModel,
                        model_params,
                        fit_params,
                        cv=make_kf,
                        folds=5)

FOLD: 1, SCORE: 278.96667
FOLD: 2, SCORE: 285.86243
FOLD: 3, SCORE: 278.37800
FOLD: 4, SCORE: 283.56761
FOLD: 5, SCORE: 276.89847
oof score: 280.73464

CPU times: user 4min 20s, sys: 3.2 s, total: 4min 23s
Wall time: 1min 18s


In [52]:
def predict(test, models):
    preds = []
    for i, (name, est) in enumerate(models.items()):
        print(f'{name}')
        preds.append(est.predict(test.values))
    preds = np.mean(preds, axis=0)
    preds = np.expm1(preds)

    return preds

In [53]:
preds = predict(test, models)

LGBM_FOLD0
LGBM_FOLD1
LGBM_FOLD2
LGBM_FOLD3
LGBM_FOLD4


In [54]:
submit_df = pd.read_csv('sample_submission.csv')

In [55]:
test_predicted = preds.reshape(-1)
test_predicted = (np.round(test_predicted/25)*25).astype(int)
submit_df["LOAN_AMOUNT"]=test_predicted
print(submit_df)

       LOAN_ID  LOAN_AMOUNT
0      2041445          375
1      1944435          375
2      2083354          475
3      1993565          550
4      2064272          275
...        ...          ...
91817  1993862          325
91818  2015070           75
91819  1950349          400
91820  1921580         1550
91821  1976733          500

[91822 rows x 2 columns]


In [56]:
submit_df.to_csv('submission.csv',index=False)